In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
from sklearn.model_selection import train_test_split
#import column transformer and one hot encoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

In [53]:
# Read in data set as DF
BankDF = pd.read_csv("bank-additional-full.csv", sep=";")
BankDF

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [54]:
# Check for missing values or null values
BankDF.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [55]:
# Missing Values
missing_values = BankDF.isna().sum()
# Print missing values
print(missing_values)

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


In [56]:
BankDF.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [57]:
# Remove unnecesssary columns
columns_to_remove = ["default", "contact", "day_of_week", "duration"]

In [58]:
# Remove the columns from the dataframe
BankDF = BankDF.drop(columns_to_remove, axis=1)
BankDF.columns

Index(['age', 'job', 'marital', 'education', 'housing', 'loan', 'month',
       'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [59]:
# Rename columns for readability reasons, columns: pdays, poutcome, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m,
#nr.employed, and y

BankDF = BankDF.rename(columns={
    'previous': 'Number_of_contacts_before_campaign', \
    'pdays': 'Days_since_last_contacted (numeric; 999 means not previously contacted)', \
    'poutcome': 'Previous_campaign_outcome (categorical: "failure","nonexistent","success")', \
    'emp.var.rate': 'Employment_variation_rate - quarterly_indicator (numeric)', \
    'cons.price.idx': 'Consumer_price_index - monthly_indicator (numeric)', \
    'cons.conf.idx': 'Consumer_confidence_index - monthly_indicator (numeric)', \
    'euribor3m': 'Euribor_3_month_rate - daily_indicator (numeric)', \
    'nr.employed': 'Number_of_employees - quarterly_indicator (numeric)', \
    'y': 'Subscribed_to_term_deposit? (binary: "yes","no")'}
)

BankDF.head(10)
BankDF['job'].value_counts()

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [60]:
# Capitolize first letter of any columns with lower case letters for first letter
BankDF = BankDF.rename(columns=lambda x: x.title())
BankDF.columns


Index(['Age', 'Job', 'Marital', 'Education', 'Housing', 'Loan', 'Month',
       'Campaign',
       'Days_Since_Last_Contacted (Numeric; 999 Means Not Previously Contacted)',
       'Number_Of_Contacts_Before_Campaign',
       'Previous_Campaign_Outcome (Categorical: "Failure","Nonexistent","Success")',
       'Employment_Variation_Rate - Quarterly_Indicator (Numeric)',
       'Consumer_Price_Index - Monthly_Indicator (Numeric)',
       'Consumer_Confidence_Index - Monthly_Indicator (Numeric)',
       'Euribor_3_Month_Rate - Daily_Indicator (Numeric)',
       'Number_Of_Employees - Quarterly_Indicator (Numeric)',
       'Subscribed_To_Term_Deposit? (Binary: "Yes","No")'],
      dtype='object')

In [61]:
# Check for missing values or null values
BankDF.isnull().sum()

Age                                                                           0
Job                                                                           0
Marital                                                                       0
Education                                                                     0
Housing                                                                       0
Loan                                                                          0
Month                                                                         0
Campaign                                                                      0
Days_Since_Last_Contacted (Numeric; 999 Means Not Previously Contacted)       0
Number_Of_Contacts_Before_Campaign                                            0
Previous_Campaign_Outcome (Categorical: "Failure","Nonexistent","Success")    0
Employment_Variation_Rate - Quarterly_Indicator (Numeric)                     0
Consumer_Price_Index - Monthly_Indicator

Begin Segmenting Customers

In [62]:
# Group by various columns and calculate the number of subscriptions for each combination
grouped = BankDF.groupby(['Age', 'Job', 'Marital', 'Education', 'Housing', 'Loan']).agg({'Subscribed_To_Term_Deposit? (Binary: "Yes","No")': 'count'})

In [74]:
# Sort the results by the number of subscriptions in descending order
grouped = grouped.sort_values(by='Subscribed_To_Term_Deposit? (Binary: "Yes","No")', ascending=False)
# Reset the index to create a flat table
grouped = grouped.drop('Subscribed_to_term_deposit? (binary: "Yes","No")', axis=1).reset_index()

# Print the top 5 combinations
print(grouped.head(10))

KeyError: '[\'Subscribed_to_term_deposit? (binary: "Yes","No")\'] not found in axis'